In [1]:
%run Fnc_StartupFabricSempy

StatementMeta(, 9a41cf85-1207-4c0b-81d2-6d2643947be7, 9, Finished, Available, Finished)

In [2]:
prm_starttime = datetime.now()
NotebookName = "Nbt_Landing_GetSMHistory"

StatementMeta(, 9a41cf85-1207-4c0b-81d2-6d2643947be7, 10, Finished, Available, Finished)

In [3]:
Table_Name = 'Landing_Fabric_SM_RefreshHistory'

StatementMeta(, 9a41cf85-1207-4c0b-81d2-6d2643947be7, 11, Finished, Available, Finished)

In [4]:
lh_abfs_path = fnc_startUp(Table_Name)

StatementMeta(, 9a41cf85-1207-4c0b-81d2-6d2643947be7, 12, Finished, Available, Finished)

niet leeg


SynapseWidget(Synapse.DataFrame, 20c974c7-0b93-4a1b-a9c6-48054c7c0f2b)

In [5]:
SemanticModels = spark.sql("""select Id, WSID
from Lkh_SemanticLink.Landing_Fabric_Items
where Type='SemanticModel' and DisplayName<>'Report Usage Metrics Model'""")

StatementMeta(, 9a41cf85-1207-4c0b-81d2-6d2643947be7, 13, Finished, Available, Finished)

In [6]:
for Id, WSID in SemanticModels.toLocalIterator():
    dataset_ID = Id
    try:
        a = sempy_labs.get_semantic_model_refresh_history(dataset = dataset_ID, workspace = WSID)     
        df = pd.DataFrame(a)
        if not df.empty: # check if the list is not empty to avoid errors
            df = pd.DataFrame(a)
            df.drop('Attempts', inplace=True, axis=1)
            df = fnc_PrepareColumns(df)
            sparkdf = spark.createDataFrame(df)
            sparkdf = sparkdf.withColumn('WSID', lit(WSID))
            sparkdf = sparkdf.withColumn('SMID', lit(dataset_ID))
            sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")                   
    except Exception as e:
        print(f"Error fetching semantic model objects for {dataset_ID}: {e}")
        continue

StatementMeta(, 9a41cf85-1207-4c0b-81d2-6d2643947be7, 14, Submitted, Running, Running)

Error fetching semantic model objects for 66c67479-aeb2-4c14-96ec-c349c08a221a: 415 Unsupported Media Type for url: https://api.fabric.microsoft.com//v1.0/myorg/groups/42e96bd0-04e2-40b5-807d-3abbeaf7d4d6/datasets/66c67479-aeb2-4c14-96ec-c349c08a221a/refreshes
Error: {"error":{"code":"InvalidRequest","message":"Invalid dataset. This API can only be called on a Model-based dataset"}}
Headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': '37ece4f3-dcac-423c-bdfc-eb5187f7cb29', 'Access-Control-Expose-Headers': 'RequestId', 'request-redirected': 'true', 'home-cluster-uri': 'https://wabi-north-europe-g-primary-redirect.analysis.windows.net/', 'Date': 'Fri, 12 Sep 2025 08:22:45 GMT'}
Error fetching semantic model objects for b9068b21-

In [ ]:
#fill list of workspaces
workspaces = fabric.list_workspaces().sort_values(by='Name', ascending=True)
df = pd.DataFrame(workspaces)

# loop through the workspaces
for label, content in df.items():
    if label == 'Id': #loop through the ID's
        for id in content:
            #lijst vullen met alle items in de werkruimte
            temp_items = fabric.list_items(workspace=id)
            itemdf = pd.DataFrame(temp_items)
            #print(itemdf)
            if not itemdf.empty: # check if the list is not empty to avoid errors
            # filter out semantic models
                df_semantic_models = itemdf[(itemdf['Type'] == 'SemanticModel') & (itemdf['Display Name'] != 'Report Usage Metrics Model')]
                #print(df_semantic_models)
                for _, row in df_semantic_models.iterrows():
                    dataset_name = row['Display Name']
                    dataset_ID = row['Id']
                    #get the relationships of each model
                    try:
                        a = sempy_labs.get_semantic_model_refresh_history(dataset = dataset_ID, workspace = id)     
                        df = pd.DataFrame(a)
                        if not df.empty: # check if the list is not empty to avoid errors
                            df = pd.DataFrame(a)
                            df.drop('Attempts', inplace=True, axis=1)
                            df = fnc_PrepareColumns(df)
                            sparkdf = spark.createDataFrame(df)
                            sparkdf = sparkdf.withColumn('WSID', lit(id))
                            sparkdf = sparkdf.withColumn('SMID', lit(dataset_ID))
                            sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")                   
                    except Exception as e:
                        print(f"Error fetching semantic model objects for {dataset_name}: {e}")
                        continue



                    

In [ ]:
prm_endtime = datetime.utcnow()
fnc_LognotebookRefresh(NotebookName, prm_starttime, prm_endtime)

StatementMeta(, , -1, Waiting, , Waiting)